# Reddit Finace Data analysis, 
## Joao P. Maia

Reddit is a major place where people discuss finance and the stock market. What r/Wallstreetbets did no GME in early 2021 is a good example.

In this notebook, I'm going to analyze and explore posts from the biggest subreddits(15k>posts)  about finance using tools written in python for data analysis and natural language processing. 

The dataset was gathered from this [Keagle dataset](https://www.kaggle.com/leukipp/reddit-finance-data). It contains posts from early 2021 until now (05/07/2021).

We are using data from those subreddits:
- r/wallstreetbets: #638158 (2021-01-01 00:02:06 - 2021-07-05 19:05:13)
- r/gme: #224149 (2021-01-01 04:08:51 - 2021-07-05 19:01:50)
- r/personalfinance: #64798 (2021-01-24 19:30:31 - 2021-07-05 19:03:36)
- r/stocks: #50729 (2021-01-01 00:05:17 - 2021-07-05 19:03:41)
- r/pennystocks: #42571 (2021-01-01 00:13:41 - 2021-07-05 19:04:28)
- r/stockmarket: #27710 (2021-01-01 02:42:42 - 2021-07-05 18:59:07)
- r/investing: #25486 (2021-01-01 00:18:40 - 2021-07-05 19:03:27)
- r/robinhoodpennystocks: #21200 (2021-01-01 00:27:36 - 2021-07-05 14:34:27)
- r/robinhoodpennystocks: #21200 (2021-01-01 00:27:36 - 2021-07-05 14:34:27)
- r/options: #18378 (2021-01-01 01:39:43 - 2021-07-05 19:00:27)


---
All analyses are made in different notebooks to keep the code organized. Each notebook is about a different term (such as GME, SILVER...). However, this file (main.py) is about cleaning and categorizing the data.

In [179]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [180]:
#Loading all IDs into a list
engine = create_engine("sqlite:///db/reddit_financial.db")
dfIDS = pd.read_sql_query(
    "SELECT id FROM reddit_data",
    con=engine,
)
dfIDS = dfIDS["id"].values

In [181]:
#Transform all csv files in dataframes
dataframes = []

for subdir, dirs, files in os.walk("./datasets"):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".csv"):
            print ("ADDED[{}] - {}".format(filepath,filepath.split(os.sep)[-2]).replace(".csv",""))
            df = pd.read_csv(filepath)

            #Cleaning deleted/removed or null posts
            df = df.dropna(subset=['selftext'])
            
            df = df[df['deleted'] !=1]
            df = df[df['removed'] !=1]

            df["selftext"] = df["selftext"].astype(str)

            #Remove if ID exists
            df = df.query('id not in @dfIDS')


            
            df['subReddit'] = filepath.split(os.sep)[-2].replace(".csv","")
            if(len(df)>0):
                dataframes.append(df)


ADDED[./datasets\gme\submissions_reddit] - gme
ADDED[./datasets\investing\submissions_reddit] - investing
ADDED[./datasets\options\submissions_reddit] - options
ADDED[./datasets\pennystocks\submissions_reddit] - pennystocks
ADDED[./datasets\personalfinance\submissions_reddit] - personalfinance
ADDED[./datasets\robinhoodpennystocks\submissions_reddit] - robinhoodpennystocks
ADDED[./datasets\stockmarket\submissions_reddit] - stockmarket
ADDED[./datasets\stocks\submissions_reddit] - stocks
ADDED[./datasets\wallstreetbets\submissions_reddit] - wallstreetbets


In [182]:
tam = 0
for df in dataframes:
   print("{} : {}".format(df["subReddit"].unique(),len(df)))
   tam += len(df)
print("Total Size: {}".format(tam))

['gme'] : 1
Total Size: 1


## Cleanning

The next step is to clear all the data from the bodies/titles to achieve better results in the sentiment analysis.

This piece of code was given by the ICA laboratory in PUC-RIO. 



In [183]:
import nltk
import spacy
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import contractions_dict
import unicodedata

In [184]:
# Tokenizer and stopword
nlp = spacy.load('en_core_web_md')
tokenizer = ToktokTokenizer()
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maiaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [185]:
# base code:
# https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/notebooks/Ch07_Analyzing_Movie_Reviews_Sentiment/text_normalizer.pys

def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

def lemmatize(text):
  text = nlp(text)
  text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
  return text

def stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

def remove_special_characters(text, remove_digits=False):
  special_char_pattern = re.compile(r'([{.(-)!}])')
  text = special_char_pattern.sub(" \\1 ", text)

  pattern = r'[^a-zA-z0-9\s]'
  text = re.sub(pattern, '', text)
  return text


def expand_contractions(text, contraction_mapping=contractions_dict):
  contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                    flags=re.IGNORECASE | re.DOTALL)

  def expand_match(contraction):
      match = contraction.group(0)
      first_char = match[0]
      expanded_contraction = contraction_mapping.get(match) \
          if contraction_mapping.get(match) \
          else contraction_mapping.get(match.lower())
      expanded_contraction = first_char + expanded_contraction[1:]
      return expanded_contraction
  
  try:
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
  except:
    return text
  return expanded_text

def remove_accent(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def strip_html_tags(text):
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"\s+", " ", text)
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [186]:
def clean(corpus):

  normalized_corpus = []
  for doc in corpus:
    doc = strip_html_tags(doc)

    doc = remove_accent(doc)

    doc = expand_contractions(doc)

    doc = doc.lower()

    doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)

    doc = lemmatize(doc)

    doc = remove_special_characters(doc)

    doc = re.sub(' +', ' ', doc)

    doc = remove_stopwords(doc)

    normalized_corpus.append(doc)

  return normalized_corpus 

In [187]:
import time
start = time.time()
''''''
for df in dataframes:
    normalized_corpus_body = clean(df.selftext.to_list())
    df.selftext = normalized_corpus_body

    normalized_corpus_title = clean(df.title.to_list())
    df.title = normalized_corpus_title
''''''

end = time.time()

total_time = (end - start)/60 # minutes
print("Cleaning Time:")
print(str(total_time) + " min")

Cleaning Time:
0.0005333383878072102 min


In [188]:
#Saving clean text on db
from sqlalchemy import create_engine
engine = create_engine("sqlite:///db/reddit_financial.db")

for df in dataframes:
    df.to_sql('reddit_clean_raw', con=engine, if_exists='replace')

## Sentiment Analisys
We are going to use the AFINN library to analyze the sentiment in both body and title. This library was trained using microblogs which are similar to Reddit in their behavior.

In [189]:
from afinn import Afinn
from sqlalchemy import create_engine,text

In [190]:
#Loading posts from the cleaned dataset
engine = create_engine("sqlite:///db/reddit_financial.db")
df_Clean = pd.read_sql_table(
    "reddit_clean_raw",
    con=engine,
    parse_dates=[
        'created',
        'retrieved',
        'edited',
    ]
)

In [191]:
df_Clean.shape
df_Clean

,index,id,author,created,retrieved,edited,pinned,archived,locked,removed,...,upvote_ratio,score,gilded,total_awards_received,num_comments,num_crossposts,selftext,thumbnail,shortlink,subReddit
0,5,krnthg,nicky94,2021-01-06 13:25:49,2021-02-28 16:51:20,2021-01-06 13:28:54,0,0,0,0,...,0.94,14,0,0,9,0,speculation idea gamestop might present overal...,self,https://redd.it/krnthg,gme


In [192]:
#Using AFINN
afinn = Afinn()
sentiment_afinn_body = []
sentiment_afinn_class_body = []

sentiment_afinn_title = []
sentiment_afinn_class_title= []

for row in df_Clean.selftext:
    afinn_score = afinn.score(row)
    sentiment_afinn_body.append(afinn_score)
    sentiment_afinn_class_body.append('positive' if afinn_score>0 else 'neutral' if afinn_score == 0 else 'negative')


for row in df_Clean.title:
    afinn_score = afinn.score(row)
    sentiment_afinn_title.append(afinn_score)
    sentiment_afinn_class_title.append('positive' if afinn_score>0 else 'neutral' if afinn_score == 0 else 'negative')


df_Clean['SentimentAFINN_body'] = sentiment_afinn_body 
df_Clean['SentimentAFINN_class_body'] = sentiment_afinn_class_body 
df_Clean['SentimentAFINN_title'] = sentiment_afinn_title 
df_Clean['SentimentAFINN_class_title'] = sentiment_afinn_class_title 

In [193]:
df_Clean.head()
df_Clean.columns

Index(['index', 'id', 'author', 'created', 'retrieved', 'edited', 'pinned',
       'archived', 'locked', 'removed', 'deleted', 'is_self', 'is_video',
       'is_original_content', 'title', 'link_flair_text', 'upvote_ratio',
       'score', 'gilded', 'total_awards_received', 'num_comments',
       'num_crossposts', 'selftext', 'thumbnail', 'shortlink', 'subReddit',
       'SentimentAFINN_body', 'SentimentAFINN_class_body',
       'SentimentAFINN_title', 'SentimentAFINN_class_title'],
      dtype='object')

In [194]:
#Putting data on a temp table
df_Clean.to_sql('reddit_data_temp', con=engine, if_exists='replace',index=False)

#Adding new IDs to main table
connection = engine.connect()
result = connection.execute("INSERT OR IGNORE INTO reddit_data SELECT id, author, created, retrieved, edited, pinned, archived, locked, removed, deleted, is_self, is_video, is_original_content, title, link_flair_text, upvote_ratio, score, gilded, total_awards_received, num_comments, num_crossposts, selftext, thumbnail, shortlink, subReddit, SentimentAFINN_body, SentimentAFINN_class_body, SentimentAFINN_title, SentimentAFINN_class_title FROM reddit_data_temp")
connection.close()

#Deleting temp table
connection = engine.connect()
result = connection.execute("DROP TABLE reddit_data_temp")
connection.close()

In [195]:
connection = engine.connect()
result = connection.execute("DELETE FROM reddit_clean_raw WHERE id IN (SELECT id from reddit_data)")
connection.close()